In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json
import string

W1122 02:38:17.698544  1124 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\tflearn\helpers\summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W1122 02:38:17.702547  1124 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\tflearn\helpers\trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W1122 02:38:17.847897  1124 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\tflearn\collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



curses is not supported on this machine (please install/reinstall curses for an optimal experience)


W1122 02:38:17.937200  1124 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\tflearn\config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W1122 02:38:18.097257  1124 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\tflearn\config.py:129: The name tf.add_to_collection is deprecated. Please use tf.compat.v1.add_to_collection instead.

W1122 02:38:18.098274  1124 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\tflearn\config.py:131: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



In [2]:
import numpy as np

In [3]:
import pickle

In [134]:
with open("intents.json") as file:
    data = json.load(file)
    
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'positive', 'patterns': ['Thanks', 'Thank you', 'Thats helpful'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'negative', 'patterns': ['this is not what', 'No', 'useless'], 'responses': ['Sorry, did I give the wrong answer?', 'Is it not what you were looking for?']}, {'tag': 'Fees', 'patterns': ['How can I find out how much I have to pay?', 'Why am I charged as a "non Quebec resident" when I am definitely a Quebec resident?', 'What if I apply for Quebec Resident status but my residency status is not approved before the fee payment deadline?', 'When are my fees 

In [135]:
words = []
labels = []
docs_x = []
docs_y = []
xx = []
for intent in data["intents"]:
        for pattern in intent["patterns"]:
            ##stemming to eliminate extra words
            ##tokeninze
            wrds = nltk.word_tokenize(pattern) ##list of words in patters
            words.extend(wrds) ##instead of append
            xx.append(pattern)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
            
words = [stemmer.stem(w.lower()) for w in words if w != "?"]  ##check the vocab size
words = sorted(list(set(words)))
labels = sorted(labels)

In [136]:
fasa = []
table = str.maketrans('', '', string.punctuation)
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [137]:
for line in xx:
    words = line.split()
    stripped = [w.translate(table) for w in words]
    #stripped = list(filter(None, stripped))
    stripped = [word.lower() for word in stripped]
    #stripped = [w for w in stripped if not w in stop_words]
    #stemmed = [porter.stem(word) for word in stripped]
    fasa.append(' '.join(stripped))
X_train = [i for i in fasa if i] 

In [138]:
X_train

['hi',
 'how are you',
 'is anyone there',
 'hello',
 'good day',
 'bye',
 'see you later',
 'goodbye',
 'thanks',
 'thank you',
 'thats helpful',
 'this is not what',
 'no',
 'useless',
 'how can i find out how much i have to pay',
 'why am i charged as a non quebec resident when i am definitely a quebec resident',
 'what if i apply for quebec resident status but my residency status is not approved before the fee payment deadline',
 'when are my fees due when must they be paid',
 'how can i pay my fees',
 'will i be charged a late payment fee or interest if i pay my fees at my bank by the deadline date',
 'what happens if i miss the payment deadline and dont pay my fees on time',
 'how are the tuition and compulsory fees charged',
 'how can i find out how much i have to pay',
 'financial support for meng in electrical and computer engineering',
 'who sets the tuition fee rates for studying in quebec',
 'do i have to pay a health insurance premium if i am an international student',
 'd

In [169]:
set(docs_y)

{'Admission',
 'Courses',
 'Fees',
 'General',
 'Health',
 'Other',
 'Refunds',
 'goodbye',
 'greeting',
 'negative',
 'positive'}

In [140]:
maxLen = len(max(X_train, key=len).split())

In [141]:
yy =np.array(docs_y)

In [142]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(yy)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
onehot_encoded

D:\MiniConda\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.

In [143]:
def read_glove_vecs(glove_file):
    with open(glove_file,encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype='float32')
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [86]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove6b200d/glove.6B.200d.txt')

In [48]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [144]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):

    m = X.shape[0]                                   # number of training examples

    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        sentence_words = X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+ 1
            
 
    return X_indices

In [145]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [146]:
len(word_to_index) + 1  

400001

In [147]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 200)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it non-trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len,emb_dim,trainable = False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [153]:
def chatter(input_shape, word_to_vec_map, word_to_index):

    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape,dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences = True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences = False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(11)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices,outputs=X)
    

    return model

In [154]:
model = chatter((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 24)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 24, 200)           80000200  
_________________________________________________________________
lstm_11 (LSTM)               (None, 24, 128)           168448    
_________________________________________________________________
dropout_11 (Dropout)         (None, 24, 128)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 11)                1419      
__________

In [155]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [156]:
tt = np.array(X_train)
X_train_indices = sentences_to_indices(tt, word_to_index, maxLen)

In [157]:
model.fit(X_train_indices, onehot_encoded, epochs = 50, batch_size = 32, shuffle=True)

W1122 03:28:23.979940  1124 deprecation.py:323] From D:\MiniConda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
51/51 [==============================] - 4s 71ms/step - loss: 2.3867 - acc: 0.0588
Epoch 2/50
51/51 [==============================] - 0s 3ms/step - loss: 2.2836 - acc: 0.2549
Epoch 3/50
51/51 [==============================] - 0s 2ms/step - loss: 2.1782 - acc: 0.2549
Epoch 4/50
51/51 [==============================] - 0s 2ms/step - loss: 2.1192 - acc: 0.2745
Epoch 5/50
51/51 [==============================] - 0s 2ms/step - loss: 1.9936 - acc: 0.2549
Epoch 6/50
51/51 [==============================] - 0s 2ms/step - loss: 2.0212 - acc: 0.3137
Epoch 7/50
51/51 [==============================] - 0s 2ms/step - loss: 1.8994 - acc: 0.4118
Epoch 8/50
51/51 [==============================] - 0s 2ms/step - loss: 1.7966 - acc: 0.4118
Epoch 9/50
51/51 [==============================] - 0s 2ms/step - loss: 1.6996 - acc: 0.4510
Epoch 10/50
51/51 [==============================] - 0s 2ms/step - loss: 1.6350 - acc: 0.4510
Epoch 11/50
51/51 [==============================] - 0s 2ms/step - l

In [160]:
def pp(strr):
    words = strr.split()
    stripped = [w.translate(table) for w in words]
    #stripped = list(filter(None, stripped))
    stripped = [word.lower() for word in stripped]
    #stripped = [w for w in stripped if not w in stop_words]
    #stemmed = [porter.stem(word) for word in stripped]
    return(' '.join(stripped))

In [167]:
def chat():
    print("start chit chat! (Enter quit to exit)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        x_test = np.array([pp(inp)])
        X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
        pred = model.predict(X_test_indices)
        num = np.argmax(pred)
        print(num)

In [170]:
chat()

start chit chat! (Enter quit to exit)
You: is tomorrow friday
8
You: no
9
You: you are wrong
8
You: quit


In [171]:
model.save('chatter.h5')